In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import random
import gc
import copy

plt.ion()  

def seed_everything(seed: int):
    # This is a utility function you can use for all your ML applications
    # More generic than you would need for this particular assignment
    gc.collect()
    torch.cuda.empty_cache()
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(57)


In [2]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
device = torch.device(device)

In [3]:
baseModel = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
print(baseModel.to(device))

Using cache found in /home/ankan/.cache/torch/hub/pytorch_vision_v0.10.0
/home/ankan/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ankan/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
from PIL import Image

image_path = 'ex.jpg'  # Replace with the path to your image
image = Image.open(image_path)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(image).unsqueeze(0)  # Add a batch dimension
input_tensor = input_tensor.to(device)

In [6]:
intermediate_outputs = {}

def hook_fn(module, input, output, layer_name):
    intermediate_outputs[layer_name] = output

# Register hooks on the desired layers to record intermediate outputs
for layer_name, layer in baseModel.named_modules():
    layer.register_forward_hook(lambda module, input, output, layer_name=layer_name: hook_fn(module, input, output, layer_name))

# Forward pass the image through the model
baseModel.eval()
output = baseModel(input_tensor)

# Access intermediate layer outputs
for layer_name, output_tensor in intermediate_outputs.items():
    print(f"Layer: {layer_name}, Output Shape: {output_tensor.shape}")

Layer: features.0, Output Shape: torch.Size([1, 64, 224, 224])
Layer: features.1, Output Shape: torch.Size([1, 64, 224, 224])
Layer: features.2, Output Shape: torch.Size([1, 64, 224, 224])
Layer: features.3, Output Shape: torch.Size([1, 64, 224, 224])
Layer: features.4, Output Shape: torch.Size([1, 64, 112, 112])
Layer: features.5, Output Shape: torch.Size([1, 128, 112, 112])
Layer: features.6, Output Shape: torch.Size([1, 128, 112, 112])
Layer: features.7, Output Shape: torch.Size([1, 128, 112, 112])
Layer: features.8, Output Shape: torch.Size([1, 128, 112, 112])
Layer: features.9, Output Shape: torch.Size([1, 128, 56, 56])
Layer: features.10, Output Shape: torch.Size([1, 256, 56, 56])
Layer: features.11, Output Shape: torch.Size([1, 256, 56, 56])
Layer: features.12, Output Shape: torch.Size([1, 256, 56, 56])
Layer: features.13, Output Shape: torch.Size([1, 256, 56, 56])
Layer: features.14, Output Shape: torch.Size([1, 256, 56, 56])
Layer: features.15, Output Shape: torch.Size([1, 256